In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

2023-01-05 17:01:43.823535: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-05 17:01:43.823600: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv("data/heart.csv")
df.pop("HeartDisease")

columns = df.columns.values
rand = random.randint(0, len(columns))

features = df.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'Age': 54,
 'Sex': 'M',
 'ChestPainType': 'ATA',
 'RestingBP': 110,
 'Cholesterol': 208,
 'FastingBS': 0,
 'RestingECG': 'Normal',
 'MaxHR': 142,
 'ExerciseAngina': 'N',
 'Oldpeak': 0.0,
 'ST_Slope': 'Up'}

In [3]:
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

In [4]:
def prepare_json(inputs: dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, int):
            feature_spec[keys] = int_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [5]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)
    
    endpoint = "https://heart-prediction.up.railway.app/v1/models/heart-disease-model:predict"
    response = requests.post(endpoint, data=json_data)
    prediction = response.json()

    prediction = response.json()["predictions"][0][0]
    
    if prediction < 0.6:
        return "Normal"
    else:
        return "Heart Disease"

In [6]:
make_predictions(inputs)

KeyError: 'predictions'